In [ ]:
import pandas as pd
import ast
import datetime
pd.set_option('display.max_row', 1000)
data = pd.read_csv("/Users/l.sergeev/Downloads/result_ios_Nav_bar_2021-01-25.csv")
data.info()

In [ ]:
data

In [ ]:
df_users = data[['idDevice','dtEvent','tsFirstLaunch','subscription_type']].drop_duplicates().dropna().reset_index(drop=True)


In [ ]:
df_users

In [ ]:
df_users['first_date'] = pd.to_datetime(df_users['tsFirstLaunch'],unit='s')

In [ ]:
df_users['first_date'] = df_users['first_date'].dt.date


In [ ]:
data_for_ret = df_users[df_users['tsFirstLaunch'] >= 1606780800]
data_for_ret

In [ ]:
data_for_ret['dtEvent'] = pd.to_datetime(data_for_ret['dtEvent'])
data_for_ret['first_date'] = pd.to_datetime(data_for_ret['first_date'])

data_for_ret

In [ ]:
data_for_ret['diff'] = (data_for_ret['dtEvent'] - data_for_ret['first_date']).dt.days
data_for_ret

In [ ]:
data_for_ret_group = data_for_ret.groupby(['diff','first_date','subscription_type'])['idDevice'].nunique().reset_index()
data_for_ret_group

In [ ]:
data_for_ret_group.to_csv("/Users/l.sergeev/Downloads/data_for_ret_group_ios.csv")

In [ ]:
retention_0 = data_for_ret[(data_for_ret['diff'] == 0)]
retention_0

In [ ]:
retention_0_group = retention_0.groupby(['diff','first_date','subscription_type'])['idDevice'].nunique().reset_index()

In [ ]:
retention_7d = data_for_ret[(data_for_ret['diff'] >= 1) & (data_for_ret['diff'] <= 30)]
retention_7d

In [ ]:
retention_7d_group = retention_7d.groupby(['first_date','subscription_type'])['idDevice'].nunique().reset_index()
retention_7d_group

In [ ]:
data_for_ret

In [ ]:
data_for_ret_active = data_for_ret[['dtEvent','idDevice','first_date','subscription_type']][data_for_ret['subscription_type'] == 'active']
data_for_ret_active

In [ ]:
data_for_ret_active_group = data_for_ret_active.groupby(['dtEvent','first_date'])['idDevice'].nunique().reset_index()
data_for_ret_active_group

In [ ]:
data_for_ret_active_group.to_csv("/Users/l.sergeev/Downloads/data_for_ret_active_group.csv")

In [ ]:
#data_for_ret_active_base = data_for_ret_active.groupby(['first_date'])['idDevice'].nunique().reset_index()
#data_for_ret_active_base

In [ ]:
data1 = pd.read_csv("/Users/l.sergeev/Downloads/3551648.customEvents.e0dc3e272b9ccec99ca801c65956996d.0.csv")
data2 = pd.read_csv("/Users/l.sergeev/Downloads/3551648.customEvents.d8d97de16a370d7aaecd6e51829cdbaf.1.csv")



In [ ]:
data1_clean = data1.drop_duplicates().dropna().reset_index(drop=True)
data2_clean = data2.drop_duplicates().dropna().reset_index(drop=True)


In [ ]:
data1_clean

In [ ]:
data2_clean

In [ ]:
new_version_users_data1 = data1_clean.loc[data1_clean.idAppVersionTitle.str.contains("5.0")].reset_index(drop=True)
new_version_users_data2 = data2_clean.loc[data2_clean.idAppVersionTitle.str.contains("5.0")].reset_index(drop=True)


In [ ]:
new_version_users_data1

In [ ]:
new_version_users_data2

In [ ]:
frames = [new_version_users_data1, new_version_users_data2]
result = pd.concat(frames)

In [ ]:
def event_parser(df):
    names = df['params.name'][0]
    names = ast.literal_eval(names)
    df['params.value'] = df['params.value'].astype(str).apply(lambda x: ast.literal_eval(x))
    df1 = pd.DataFrame(df["params.value"].to_list(), columns=names)
    df2 = df.join(df1)
    return df2

In [ ]:
result_parsed = event_parser(result.head(10000000))

In [ ]:
result

In [ ]:
result_5_0_1 = result.loc[result.idAppVersionTitle.str.contains("5.0.1")].reset_index(drop=True)


In [ ]:
result_parsed = event_parser(result_5_0_1)

In [ ]:
result_parsed

In [ ]:
df_users_iOS = result_parsed[['idDevice','dtEvent','tsFirstLaunch','idAppVersionTitle','subscription_type']].drop_duplicates().dropna().reset_index(drop=True)


In [ ]:
df_users_iOS['platform'] = 'iOS'
df_users_iOS

In [ ]:
df_users_android = data[['idDevice','dtEvent','tsFirstLaunch','idAppVersionTitle','subscription_type']].drop_duplicates().dropna().reset_index(drop=True)
df_users_android['platform'] = 'Android'
df_users_android

In [ ]:
frames2 = [df_users_iOS, df_users_android]
df_users_all_platforms = pd.concat(frames2)

In [ ]:
df_users_all_platforms

In [ ]:
df_users_all_platforms['first_date'] = pd.to_datetime(df_users_all_platforms['tsFirstLaunch'],unit='s')
df_users_all_platforms['first_date'] = df_users_all_platforms['first_date'].dt.date
df_users_all_platforms

In [ ]:
data_for_ret_active = df_users_all_platforms[['dtEvent','idDevice','first_date','subscription_type']][df_users_all_platforms['subscription_type'] == 'active']
data_for_ret_active

In [ ]:
data_for_ret_active_group = data_for_ret_active.groupby(['dtEvent','first_date'])['idDevice'].nunique().reset_index()
data_for_ret_active_group

In [ ]:
data_for_ret_active_group.to_csv("/Users/l.sergeev/Downloads/data_for_ret_active_group_all.csv")

In [ ]:
data_for_ret_free = df_users_all_platforms[['dtEvent','idDevice','first_date','subscription_type']][df_users_all_platforms['subscription_type'] == 'none']
data_for_ret_free

In [ ]:
data_for_ret_free_group = data_for_ret_free.groupby(['dtEvent','first_date'])['idDevice'].nunique().reset_index()
data_for_ret_free_group

In [ ]:
data_for_ret_free_group.to_csv("/Users/l.sergeev/Downloads/data_for_ret_free_group.csv")

In [ ]:
df_users_ret_all_platforms = df_users_all_platforms.groupby(['dtEvent','first_date','platform'])['idDevice'].nunique().reset_index()
df_users_ret_all_platforms

In [ ]:
df_users_ret_all_platforms.to_csv("/Users/l.sergeev/Downloads/df_users_ret_all_platforms_w_platform.csv")

In [ ]:
data_for_ret_active_ts = df_users_all_platforms[['dtEvent','idDevice','first_date','subscription_type','tsFirstLaunch']][df_users_all_platforms['subscription_type'] != 'expired']
data_for_ret_active_ts

In [ ]:
test = data_for_ret_active_ts[data_for_ret_active_ts['tsFirstLaunch'] >= 1607644800]
test

In [ ]:
test.to_csv("/Users/l.sergeev/Downloads/test_date.csv")

In [ ]:
test2 = pd.read_csv("/Users/l.sergeev/Downloads/test_date.csv")
test2

In [ ]:
test2['dtEvent'] = pd.to_datetime(test2['dtEvent'])
test2['first_date'] = pd.to_datetime(test2['first_date'])

test2

In [ ]:
test2['diff'] = (test2['dtEvent'] - test2['first_date']).dt.days


In [ ]:
test2

In [ ]:
test2_below7 = test2[(test2['diff'] >= 1) & (test2['diff'] <= 30)]
test2_below7

In [ ]:
test2_active_group = test2_below7.groupby(['first_date'])['idDevice'].nunique().reset_index()
test2_active_group

In [ ]:
test2_active_group.to_csv("/Users/l.sergeev/Downloads/test2_active_group.csv")